In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="")
project = rf.workspace("tossit").project("tossit-plates")
version = project.version(1)
dataset = version.download("yolov11")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to tossit-plates-1 in yolov11:: 100%|███████| 1394/1394 [00:00<00:00, 5621.86it/s]


In [1]:
import os
import json
import base64
from openai import OpenAI


In [ ]:
# === Configuration ===
API_KEY = ""
MODEL_NAME = "gpt-4o"
PROMPT = (
    "Calculate the waste of the plate (a number between 0 and 100, where 100 means full plate and 0 means empty). "
    "The formula is: waste = FoodArea / (plateArea - GarbageArea). "
    "FoodArea: area covered by food items; "
    "plateArea: total area of the white plate; "
    "GarbageArea: area covered by garbage (not food). "
    "Respond only with the numeric result."
)


In [4]:
# === Initialize OpenAI Client ===
client = OpenAI(api_key=API_KEY)



In [5]:


# === Utilities ===
def is_valid_image(filename):
    return filename.lower().endswith((".jpg", ".jpeg", ".png"))

def is_valid_integer_response(s):
    try:
        value = float(s)
        return value.is_integer() and 0 <= value <= 100
    except:
        return False

def call_model_on_image(image_path):
    try:
        with open(image_path, "rb") as f:
            base64_image = base64.b64encode(f.read()).decode("utf-8")

        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": PROMPT},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/jpeg;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ],
            max_tokens=50,
        )

        result_text = response.choices[0].message.content.strip()
        return {"image": os.path.basename(image_path), "result": result_text}
    except Exception as e:
        return {"image": os.path.basename(image_path), "result": None, "error": str(e)}


def parse_result(entry):
    try:
        raw = entry["result"]
        if is_valid_integer_response(raw):
            return {
                "image": entry["image"],
                "waste": int(float(raw))
            }
        else:
            raise ValueError("Not a valid integer between 0 and 100")
    except Exception as e:
        return {
            "image": entry["image"],
            "waste": None,
            "error": str(e)
        }

def save_json(data, filename):
    with open(filename, "w") as f:
        json.dump(data, f, indent=2)


In [6]:
dataset_path = "tossit-plates-1"
directory = os.path.join(os.getcwd(), dataset_path, "test", "images")
dataset = os.path.basename(dataset_path)  # 'tossit-plates-1'


In [7]:

file_list = os.listdir(directory)

In [8]:
file_list

['Image_100_jpg.rf.4c60f972743538746bcb40f6375f3c84.jpg',
 'Image_113_jpg.rf.a74d65b8c3820bf1fc0a5042f49a8419.jpg',
 'Image_124_jpeg.rf.498262980563cdd034096b600f106bfe.jpg',
 'Image_129_jpg.rf.8f4672a27e948c44543a37a699ab662a.jpg',
 'Image_13_png.rf.4eca2d77a288029ac65d067e3e2c56e5.jpg',
 'Image_18_jpg.rf.7a1991219aca1fbfb751f498b65c28a0.jpg',
 'Image_19_jpg.rf.d75fbf21860bc98018b2a63233be7fd4.jpg',
 'Image_5_jpg.rf.c195e7ba10025bf61c28d0b100edddbc.jpg',
 'Image_65_png.rf.16e299eb1513f3f02f534eb0bb328b31.jpg',
 'Image_85_jpg.rf.df6cb426c92142d009c0d11a8925e590.jpg',
 'Image_89_jpg.rf.a420b8fb1cd5aaa554bcea80a03cc763.jpg',
 'Image_92_jpg.rf.255486843921bb0e2639a4b34df155a5.jpg',
 'color_image_4_jpg.rf.2f42b15022ec8ef205353618cd69fcb6.jpg',
 'color_image_5_jpg.rf.2ec3f26f0d30388bb53b3f504f463831.jpg']

In [9]:

# === Process all images in the current folder ===
all_results = []
errors_upload = []

for filename in file_list:
    if is_valid_image(filename):
        image_path = os.path.join(directory, filename)
        result = call_model_on_image(image_path)
        all_results.append(result)
        if result.get("error") or result["result"] is None:
            errors_upload.append(result)


save_json(all_results, f"{dataset}_openai_waste_predictions.json")
save_json(errors_upload, f"{dataset}_openai_waste_errors.json")


In [10]:

# === Parse and filter model outputs ===
valid = []
invalid = []

for entry in all_results:
    parsed = parse_result(entry)
    if parsed["waste"] is not None:
        valid.append(parsed)
    else:
        invalid.append(parsed)

save_json(valid, f"{dataset}_openai_waste_cleaned.json")
save_json(invalid, f"{dataset}_openai_waste_invalid.json")

# === Summary Output ===
print("✅ Processing complete.")
print(f"Total images: {len(all_results)}")
print(f"Valid results: {len(valid)}")
print(f"Invalid results: {len(invalid)}")
print(f"Upload/API errors: {len(errors_upload)}")


✅ Processing complete.
Total images: 2
Valid results: 0
Invalid results: 2
Upload/API errors: 2


## Dataset 2 

In [9]:
# add name of the 2 dataset 
dataset_path = "dataset_name"
directory = os.path.join(os.getcwd(), dataset_path, "test", "images")
dataset = os.path.basename(dataset_path)  


In [ ]:

file_list = os.listdir(directory)

In [10]:

# === Process all images in the current folder ===
all_results = []
errors_upload = []

for filename in file_list:
    if is_valid_image(filename):
        image_path = os.path.join(directory, filename)
        result = call_model_on_image(image_path)
        all_results.append(result)
        if result.get("error") or result["result"] is None:
            errors_upload.append(result)


save_json(all_results, f"{dataset}_openai_waste_predictions.json")
save_json(errors_upload, f"{dataset}_openai_waste_errors.json")


'test'

In [ ]:

# === Parse and filter model outputs ===
valid = []
invalid = []

for entry in all_results:
    parsed = parse_result(entry)
    if parsed["waste"] is not None:
        valid.append(parsed)
    else:
        invalid.append(parsed)

save_json(valid, f"{dataset}_openai_waste_cleaned.json")
save_json(invalid, f"{dataset}_openai_waste_invalid.json")

# === Summary Output ===
print("✅ Processing complete.")
print(f"Total images: {len(all_results)}")
print(f"Valid results: {len(valid)}")
print(f"Invalid results: {len(invalid)}")
print(f"Upload/API errors: {len(errors_upload)}")


## 3 Dataset 

In [ ]:
# add name of the 2 dataset 
dataset_path = "dataset_name"
directory = os.path.join(os.getcwd(), dataset_path, "test", "images")
dataset = os.path.basename(dataset_path)  

file_list = os.listdir(directory)

In [ ]:

# === Process all images in the current folder ===
all_results = []
errors_upload = []

for filename in file_list:
    if is_valid_image(filename):
        image_path = os.path.join(directory, filename)
        result = call_model_on_image(image_path)
        all_results.append(result)
        if result.get("error") or result["result"] is None:
            errors_upload.append(result)


save_json(all_results, f"{dataset}_openai_waste_predictions.json")
save_json(errors_upload, f"{dataset}_openai_waste_errors.json")


In [ ]:

# === Parse and filter model outputs ===
valid = []
invalid = []

for entry in all_results:
    parsed = parse_result(entry)
    if parsed["waste"] is not None:
        valid.append(parsed)
    else:
        invalid.append(parsed)

save_json(valid, f"{dataset}_openai_waste_cleaned.json")
save_json(invalid, f"{dataset}_openai_waste_invalid.json")

# === Summary Output ===
print("✅ Processing complete.")
print(f"Total images: {len(all_results)}")
print(f"Valid results: {len(valid)}")
print(f"Invalid results: {len(invalid)}")
print(f"Upload/API errors: {len(errors_upload)}")
